In [1]:
from CheckEnv import check_env
from CheckPlayer import check_agent
from CheckEnvAgent import CreateFolder, load_module_player, add_game_to_syspath, setup_game, save_json
from AutoTrain import train_agent_by_level

from setup import SHOT_PATH, time_run_game, N_AGENT, N_GAME, PASS_LEVEL
import pandas as pd
import importlib.util
import numpy as np
import sys, time, shutil
import json, os, itertools



In [13]:
# dict_level_all = {'Level max': 1}
# level = 1
# if str(level) not in dict_level_all:
#     dict_level_all[level] = {'Can Train': "False",
#                             'Agents Name': []}
#     save_json(f'{SHOT_PATH}Log/level_game_all.json', dict_level_all)



In [4]:
#Thêm file temp tạm lưu cho hệ thống
def CreateTempData(env_name, agent_names, level):
    add_game_to_syspath(env_name)
    for agent_name in agent_names:
        __p0__ = load_module_player(agent_name)
        path_save_player = CreateFolder(agent_name, env_name, level)
        np.save(f'{path_save_player}Train.npy',__p0__.DataAgent())

env_name = 'Splendor_v2' 
level =  1
agent_names =    [
   "TimeBasedAlgorithm",
   "An",
   "BiasAlgorihm",
   "MultiDimensionAlgorithm",
   "StateBasedAlgorithm",
   "Trang"
  ]
CreateTempData(env_name, agent_names, level)

# Copy các agent còn thiếu cho thiếu

In [10]:
def get_level(): #Hàm này để check xem đã có đủ agent để lập một level mới hay chưa
    dict_level_all = json.load(open(f'{SHOT_PATH}Log/level_game_all.json'))
    level = dict_level_all['Level max']
    #Khi có agent mới mà đủ điều kiện thì thêm luôn vào đây
    #Đang chờ
    if str(level) in dict_level_all:
        if len(dict_level_all[str(level)]['Agents Name']) >= N_AGENT: #Thêm level mới khi có level cũ đã đủ số lượng agent
            dict_level_all[str(level)]['Can Train'] = "True"
            dict_level_all['Level max'] += 1
            dict_level_all[level+1] = {'Can Train': "False",
                                        'Agents Name': []}
        save_json(f'{SHOT_PATH}Log/level_game_all.json', dict_level_all)

def CopyAgent(agent_old, agent_new): #Tên folder của người chơi
    path_data = f'{SHOT_PATH}Agent/{agent_new}/'
    if not os.path.exists(path_data):
        os.mkdir(path_data)
    time.sleep(0.5)
    shutil.copy2(f'{SHOT_PATH}Agent/{agent_old}/Agent_player.py', f'{SHOT_PATH}Agent/{agent_new}/Agent_player.py')

#List agent ở trong một game thì xếp thứ tự theo rank
def check_copy_agent_need(env_name, level): #Khi không đủ agent để làm lv1 thì lấy thêm một agent train lại và làm level của game đó
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    add_game_to_syspath(env_name)
    __env__  = setup_game(env_name)
    current_agent_level = dict_level[env_name][str(level)][2]
    if (__env__.getAgentSize()-1) > len(current_agent_level) and len(current_agent_level) > 0: #Nếu số agent cần nhiều hơn số agent đang có của game này
        count_agent_need = (__env__.getAgentSize()-1) - len(current_agent_level)
        for id_need in range(count_agent_need):
            if id_need >= len(current_agent_level):
                id_need = 0
            id_copy = 0
            while True:
                name_agent_copy = f'{current_agent_level[id_need]}{id_copy}'
                if name_agent_copy not in dict_level[env_name][str(level)][2]:
                    dict_level[env_name][str(level)][2].append(name_agent_copy)
                    dict_level[env_name][str(level)][0].append(0)
                    dict_level[env_name][str(level)][1].append(0)

                    CopyAgent(current_agent_level[id_need], name_agent_copy)
                    break
                id_copy += 1
        save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)

def train_copy_agent(env_name, level, type_train): #1 là đã train, 0 là chưa train, -1 là đang train
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    level_train = level - 1
    add_game_to_syspath(env_name)
    __env__  = setup_game(env_name)
    if type_train == 'Train Copy':
        id_state_save = 0 
        id_win_score_save = 1
        id_agent_name_save = 2
    if type_train == 'Train New':
        id_state_save = 3
        id_win_score_save = 4
        id_agent_name_save = 5
    current_agent_level = dict_level[env_name][str(level)][id_agent_name_save]
    state_agent_level = dict_level[env_name][str(level)][id_state_save]
    for id_agent, state_agent in enumerate(state_agent_level):
        if state_agent == 0: #Chưa train
            print(id_agent, state_agent)
            agent_name = current_agent_level[id_agent]
            __p0__ = load_module_player(agent_name)
            dict_level[env_name][str(level)][id_state_save][id_agent] = -1 #Đang train
            save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)

            path_save_agent_copy = CreateFolder(agent_name, env_name, level)
            print('train_copy_agent', __env__, __p0__, __p0__.Train)
            win = train_agent_by_level(__env__, path_save_agent_copy, level_train, __p0__, 100000000000000)

            dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
            dict_level[env_name][str(level)][id_state_save][id_agent] = 1 #Đã train xong
            dict_level[env_name][str(level)][id_win_score_save][id_agent] = win
            save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)
            break


def get_new_level_for_env(): #Thêm level mới cho hệ thống khi đã đủ điều kiện
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    dict_level_all = json.load(open(f'{SHOT_PATH}Log/level_game_all.json'))

    for env_name in dict_level:
        LMAX_env = dict_level[env_name]["level_max"]+1

        if str(LMAX_env) in dict_level[env_name]:
            if len(dict_level[env_name][str(LMAX_env)][0]) > 0:
                print(dict_level[env_name]["level_max"])
                if min(dict_level[env_name][str(LMAX_env)][0]) == 1: #Khi đã đủ điều kiện lập level mới
                    __env__ = setup_game(env_name)
                    if (__env__.getAgentSize() - 1) <= len(dict_level[env_name][str(LMAX_env)][0]):
                        dict_level[env_name]["level_max"] = LMAX_env
                        save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)

        dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
        LMAX_env = dict_level[env_name]["level_max"]
        LMAX_env_new = LMAX_env + 1

        if str(LMAX_env) in dict_level[env_name]:
            if min(dict_level[env_name][str(LMAX_env)][0]) == 1:
                if LMAX_env_new <= dict_level_all["Level max"] and (str(LMAX_env_new) not in dict_level[env_name]): 
                    if dict_level_all[str(LMAX_env_new)]['Can Train'] == 'True': #Khi lưu trữ đã đủ cho level này
                        __env__ = setup_game(env_name)
                        if (__env__.getAgentSize() - 1) <= len(dict_level[env_name][str(LMAX_env)][0]):
                            lst_agent_of_level = dict_level_all[str(LMAX_env_new)]['Agents Name']
                            lst_score_agent_of_level = [0 for i in range(N_AGENT)]
                            lst_state_agent_of_level = [0 for i in range(N_AGENT)]
                            dict_level[env_name][str(LMAX_env_new)] = [[],[],[],lst_state_agent_of_level,lst_score_agent_of_level,lst_agent_of_level]
                            save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)
            
def train_all_agent_of_level(): #Train test các agent của level đó
    dict_level_all = json.load(open(f'{SHOT_PATH}Log/level_game_all.json'))
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    for env_name in dict_level:
        LMAX_env = dict_level[env_name]["level_max"]+1
        if str(LMAX_env) in dict_level[env_name]: #Có 2 loại, train N_agent để chọn ra top agent và train, copy các agent thiếu
            if LMAX_env == 1:
                type_train = 'Train Copy'
                check_copy_agent_need(env_name, LMAX_env)
                train_copy_agent(env_name, LMAX_env, type_train)
                break
            else:
                if min(dict_level[env_name][str(LMAX_env)][3]) == 1: # dùng copy agent thiếu
                    type_train = 'Train Copy'
                    check_copy_agent_need(env_name, LMAX_env)
                    train_copy_agent(env_name, LMAX_env, type_train)
                else: #Train agent lưu trữ, chọn ra agent đủ điều kiện
                    type_train = 'Train New'
                    train_copy_agent(env_name, LMAX_env, type_train)
                    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
                    if min(dict_level[env_name][str(LMAX_env)][3]) == 1:
                        lst_agent_of_level = np.array(dict_level[env_name][str(LMAX_env)][5])
                        lst_score_agent_of_level = np.array(dict_level[env_name][str(LMAX_env)][4])

                        id_agent_pass_level, = np.where(lst_score_agent_of_level > PASS_LEVEL)
                        if len(id_agent_pass_level) > 0:
                            id_agent_pass_level_sort = id_agent_pass_level[np.argsort(lst_score_agent_of_level[id_agent_pass_level])][::-1]
                            
                            dict_level[env_name][str(LMAX_env)][2] = list(lst_agent_of_level[id_agent_pass_level_sort])
                            dict_level[env_name][str(LMAX_env)][1] = [int(i) for i in lst_score_agent_of_level[id_agent_pass_level_sort]]
                            dict_level[env_name][str(LMAX_env)][0] = [1 for i in range(len(id_agent_pass_level))] #Gán thành đã train

                            save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)
                break

env_name = 'Splendor_v2'
level = 1

for i in range(1):
    get_level()
    get_new_level_for_env()
    train_all_agent_of_level()


1


# Hàm thi đâu tổ hợp đã done
Cần sửa ở trong hệ thống mỗi game thành level 0, và path đọc các file sửa

In [ ]:
import itertools
env_name = 'SushiGo'
list_name_agent = ['An', 'BiasAlgorihm', 'MultiDimensionAlgorithm', 'StateBasedAlgorithm', 'TimeBasedAlgorithm', 'Trang']
add_game_to_syspath(env_name)
__env__ = setup_game(env_name)
dict_level = {}
combinations = []
dict_result = {}
for agent in list_name_agent:
    list_name_agent_copy = list_name_agent.copy()
    list_name_agent_copy.remove(agent)
    dict_level[env_name] = {}
    p0 = load_module_player(agent).Test
    combis = enumerate(itertools.combinations(list_name_agent_copy, __env__.getAgentSize()-1))

    level_max = 1
    for level0, combination in combis:
        list_level = [0, 0, list(combination)]
        dict_level[env_name][str(level0+1)] = list_level
        level_max += 1
    
    save_json(f'{SHOT_PATH}Log/level_game{env_name}.json', dict_level)
    per_player = list(np.load(f'{SHOT_PATH}Agent/{agent}/Data/{env_name}_0/Train.npy',allow_pickle=True))
    win_total_player = 0
    for level in range(level_max):
        win, per = __env__.numba_main_2(p0, 100, per_player, level)
        win_total_player += win
        print(level, win)
    print(agent, win_total_player)
    dict_result[agent] = win_total_player
print(dict_result)